In [1]:
import subprocess
import os
from ase.io import read
from ase.io.cube import read_cube
from ase.visualize import view
from ase.io import write
from ase.build import surface
import ase.calculators.castep
import ase.io.castep
import pandas as pd
from ase import Atom
from ase import Atoms

from pymatgen.core.structure import Structure
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.symmetry.kpath import KPathSetyawanCurtarolo,KPathLatimerMunro
from pymatgen.electronic_structure.dos import *
from pymatgen.electronic_structure.plotter import BSPlotter, DosPlotter
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from python_functions import *
from bs_plotting import *
from castep_output_class import *
from optados_output_class import *
import json
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from python_functions import *
from calculation_class import *
from constants import *
#from ../../modules_codes/script_share/python/bs_plotter_felix_mod import *


In [29]:
interface = read('./structures/Megghi/AIMD-pos-1.xyz',index='5000:')

In [25]:
junction = read('./structures/Megghi/data/2Au_AIMD-pos-1.xyz',index='31000::100')

In [26]:
for frame in junction:
    # 15.281   26.467   31.110
    frame.set_cell([[15.281,0,0],[0,26.467,0],[0,0,31.110]])
    frame.set_pbc(True)
    frame.wrap()

In [27]:
view(junction)

<Popen: returncode: None args: ['c:\\Users\\fcm19\\Miniconda3\\envs\\matchem...>

In [24]:
write('./structures/Megghi/data/2Au_AIMD-wire_break_subsample_2.xyz',junction)

In [30]:
for frame in interface:
    frame.set_cell([[11.375715271,0,0],[0,14.777907317,0],[0,0,46]])
    frame.set_pbc(True)
    frame.wrap()

In [29]:
atoms = read('./structures/Megghi/data/AIMD-pos-1.xyz',index='5000::5')
for frame in atoms:
    frame.set_cell([[11.375715271,0,0],[0,14.777907317,0],[0,0,46]])
    frame.set_pbc(True)
    frame.wrap()
# print(atoms)
rot = '-180x,-90y,0z' # found using ag: 'view -> rotate'
bbox = [-10, -1, 25, 35]
# Common kwargs for eps, png, pov
generic_projection_settings = {
    'rotation': rot, # text string with rotation (default='' )
    # 'bbox' : bbox,
    'radii': .85, # float, or a list with one float per atom
    'colors': None, # List: one (r, g, b) tuple per atom
    'show_unit_cell': 2, # 0, 1, or 2 to not show, show, and show all of cell
}
# Extra kwargs only available for povray (All units in angstrom)
povray_settings = {
    'display': False, # Display while rendering
    'pause': True, # Pause when done rendering (only if display)
    'transparent': False, # Transparent background
    #'canvas_width': None, # Width of canvas in pixels
    'canvas_width': 1080,
    'canvas_height': None, # Height of canvas in pixels
    'camera_dist': 100, # Distance from camera to front atom
    'image_plane': None, # Distance from front atom to image plane
    'camera_type': 'orthographic', # perspective, ultra_wide_angle
    'point_lights': [], # [[loc1, color1], [loc2, color2],...]
    'area_light': [(2., 3., 40.), # location
    'White', # color
    .7, .7, 3, 3], # width, height, Nlamps_x, Nlamps_y
    'background': 'White', # color
    # 'textures': None, # Length of atoms list of texture names
    'textures': len(atoms[0])*['intermediate'],
    'transmittances':len(atoms[0])*[0.0],
    'celllinewidth': .05, # Radius of the cylinders representing the cell
}
# Write the .pov (and .ini) file.
# comment out render not call the povray executable
# os.system('mkdir ./pt_water_bilayer/frames2/')


In [30]:
atoms[0].get_atomic_numbers()

array([78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78,
       78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78,
       78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78,
       78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78,
       78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78,
       78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78,
       78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78,
       78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78,
       78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78,
       78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78,
       78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78, 78,
       78, 78, 78, 78, 78,  8,  1,  1,  8,  1,  1,  8,  1,  1,  8,  1,  1,
        8,  1,  1,  8,  1,  1,  8,  1,  1,  8,  1,  1,  8,  1,  1,  8,  1,
        1,  8,  1,  1,  8

In [12]:
transmit = np.zeros(len(atoms[0]))
for idx,atom in enumerate(atoms):
    i = 0
    for atom in frame.get_atomic_numbers():
        if 16.5 < atom[2] < 29.8: 
            transmit[i] = 0.8
        else: 
            transmit[i] = 0.0
        i += 1
    povray_settings['transmittances'] = transmit
    renderer = write(f'./structures/Megghi/interface/interface_{idx:04d}.pov', atoms[idx], **generic_projection_settings, povray_settings=povray_settings)
    renderer.render(povray_executable='C:/Program Files/POV-Ray/v3.7/bin/pvengine.exe')

RuntimeError: Povray left no output PNG file "c:\Users\fcm19\OneDrive - Imperial College London\Desktop\PhD\Project\photoemission\structures\Megghi\interface\interface_0000.png"

In [23]:
os.system("cd ./structures/Megghi/interface/")
print(os.system('pwd'))
os.system("C:/Program Files/POV-Ray/v3.7/bin/pvengine.exe interface_0000.ini")

1


1

In [20]:
cell = [11.375715271,14.777907317,46]
for frame in interface:
    position = frame.get_positions()
    for idx,atom in enumerate(frame.get_positions()):
        temp = [0,0,0]
        # print(atom)
        position[idx] = np.mod(atom, cell)
        # print(np.mod(atom, cell))
    print(position)
    frame.set_positions(position)

[[1.49425000e-01 9.11500000e-03 5.09677000e+00]
 [2.97461800e+00 1.17601000e-01 5.00000000e+00]
 [5.79384100e+00 2.84400000e-02 5.10078000e+00]
 ...
 [5.44036066e+00 6.72161659e-01 2.21138605e+01]
 [5.68128278e+00 8.91850657e-01 2.12190824e+01]
 [5.04582699e+00 1.52316737e+00 2.24986997e+01]]


TypeError: 'numpy.ndarray' object is not callable

In [31]:
view(interface)

<Popen: returncode: None args: ['c:\\Users\\fcm19\\Miniconda3\\envs\\matchem...>

In [7]:
interface.wrap()

AttributeError: 'list' object has no attribute 'wrap'